## Embedding
- 언어의 계층적 관계를 표현하기 위해 dense vector로 표현

### Dimension Reduction
- hidden layer는 y를 구하기 위해 x의 필요정보를 작은차원으로 압축

## Word Embedding
- 단어를 연속적인 값으로 표현하려는 시도

### 언어의 특징
- 다의어, Polysemy: 한 단어가 여러 의미
- 동형어, Homonym: 다른 의미의 한 단어
- => Word Sense Disambiguation이 필요
- 동의어, Synonyms: 한 의미의 다른 단어
- Hypernym, Hyponym: 계층구조(동물 / 원숭이)
- => 이러한 특징들은 one-hot 인코딩을 통해 반영할 수 없음

### WordNet
- 위의 특징들을 반영하기 위한 단어사전, Thesaurus
- 동의어, 상위어, 하위어에 대한 정보가 구축되어있는 Directed Acyclic Graph
- NLTK 패키지에 포함
- Korean WordNet -> KorLex, Korean WordNet
- 계층 구조의 Graph에서 단어 사이의 거리를 측정 / sim(w, w') = -log distance(w, w')
- Corpus(training 데이터) 없이도 단어 사이의 거리 측정이 가능
- cons
  - 특정 도메인 특화 수치 계산이 어려움
  - 신조어나 사전에 등록되어 있지 않은 단어 분석이 불가능

In [3]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /home/jcwee/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/jcwee/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [7]:
wn.synsets('people')

[Synset('people.n.01'),
 Synset('citizenry.n.01'),
 Synset('people.n.03'),
 Synset('multitude.n.03'),
 Synset('people.v.01'),
 Synset('people.v.02')]

In [9]:
wn.synsets('people')[0].hypernyms()

[Synset('group.n.01')]

In [10]:
def hypernyms(word):
    current_node = wn.synsets(word)[0]
    yield current_node

    while True:
        try:
            current_node = current_node.hypernyms()[0]
            yield current_node
        except IndexError:
            break
for h in hypernyms('policeman'):
    print(h)

Synset('policeman.n.01')
Synset('lawman.n.01')
Synset('defender.n.01')
Synset('preserver.n.03')
Synset('person.n.01')
Synset('causal_agent.n.01')
Synset('physical_entity.n.01')
Synset('entity.n.01')


In [11]:
[h for h in hypernyms('firefighter')]

[Synset('fireman.n.04'),
 Synset('defender.n.01'),
 Synset('preserver.n.03'),
 Synset('person.n.01'),
 Synset('causal_agent.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

In [12]:
[h for h in hypernyms('sheriff')]

[Synset('sheriff.n.01'),
 Synset('lawman.n.01'),
 Synset('defender.n.01'),
 Synset('preserver.n.03'),
 Synset('person.n.01'),
 Synset('causal_agent.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

In [13]:
[h for h in hypernyms('mailman')]

[Synset('mailman.n.01'),
 Synset('deliveryman.n.01'),
 Synset('employee.n.01'),
 Synset('worker.n.01'),
 Synset('person.n.01'),
 Synset('causal_agent.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

In [14]:
def distance(word1, word2):
    word1_hypernyms = [h for h in hypernyms(word1)]

    for i, word2_hypernym in enumerate(hypernyms(word2)):
        try :
            return i + word1_hypernyms.index(word2_hypernym)

        except ValueError:
            continue

distance('sheriff', 'student')

6

In [16]:
import numpy as np

def similarity(word1, word2):
    return -np.log(distance(word1, word2))

print(similarity('sheriff', 'student'))
print(similarity('sheriff', 'policeman'))

-1.791759469228055
-0.6931471805599453


## Data-driven Method
- 데이터가 충분하다면 task에 특화된 활용이 가능
### TF-IDF
- 단어 w의 문서 d 내에서의 중요성을 나타냄
- TF(w, d) / DF(w)
- TF(Term Frequency)
  - 단어의 문서 내 출현 횟수
  - 숫자가 클수록 중요단어
- IDF(Inverse Document Frequency)
  - 해당 단어가 출현한 문서 숫자의 역수
  - 값이 클수록 일반적으로 많이 쓰이는 단어
- 단어의 각 문서에서의 중요도를 feature로 vector화
- TF-IDF MAtrix, 단어의 각 문서별 TF-IDF수치를 vector화
  - row: 단어, column: 문서
### Co-occurrence, Based on Context Window
- 함께 나타나는 단어들을 활용
- context window를 사용하여 windowing을 실행
  - 특정 단어 주변에 나타나는 단어를 비교하여 단어간의 유사도 측정

#### 여전히 sparse vector로 표현되는 단점이 존재

## Similarity Metrics
- vector간의 거리를 구하는 방법  
- Manhattan Distance: dL1(w, v) = sigma(1~d) |wi - vi|
- Euclidean Distance: dL2(w, v) = sqrt(sigma(1~d) (wi - vi) ^ 2)
- Infinity Norm: max(|w1 - v1|, |w2 - v2|, ... )
- Cosine Similarity: cos_sim(w, v) = (w, v).dot / |w||v|  
                                   = sigma(wi*vi) / sqrt(sigma(wi^2)) sqrt(sigma(vi^2))  
- L1, L2, Infinity는 강조하고자 하는 내용에 따라 사용
- Cosine Similarity는 벡터의 방향을 중요시함

In [17]:
import pandas as pd
from collections import defaultdict

In [20]:
with open('review.sorted.uniq.refined.tok.tsv') as f:
    lines = [l.strip() for l in f.read().splitlines() if l.strip()]

In [25]:
lines

['negative\t!',
 'negative\t! 다 녹 아서 왓 어요 . . 짜증',
 'negative\t!!!!!!!!!!!!!',
 'negative\t!!!! 이게 뭐 야 ? 진짜 이게 뭐 야 ? 리뷰 보고 구매 했 는데 ! 나원참 !',
 'negative\t" 싼 게 비지떡 ". 200 ml 라고 판매 하 는데 내 가 보 기 엔 절대 이거 100 ml 도 안 된다 . 뚜껑 열 다가 손톱 부러지 고 , 향기 는 에프 킬 라 . . 구매 후기 잘 안 쓰 는데 이건 진짜 최악 이 다 .',
 'negative\t" 축 결혼 " 이 라는 리본 도 없 고 화환 하나 에 두 사람 이름 만 있 는 걸 보 니까 화 가 났 습니다 . 근처 에 화환 몇 개 놓 을 자리 도 있 었 는데 ?',
 'negative\t" 가죽 검정 " 이 라는 상품명 에서 누구 나 알 수 있 듯 인조 가죽 느낌 의 재질 입니다 . 비닐 에 가까운 재질 이 라 진짜 인조 가 죽이 라도 저렴 한 소재 일 것 같 습니다 . 이 가격 에 사 는 사람 은 소재 에 대한 기대 는 없 을 테 지만 참고 하 시 기 바랍니다 . 패드 뒷 면 은 전면 이 양면 테이프 로 부착 되 어 있 으며 그대로 신발 에 접착 시키 는 방식 입니다 . 하지만 양면 테이프 가 강해 보이 지 않 습니다 . 신발 이 조금 만 고급 이 라도 주문 하 지 마세요 . 패드 야 며칠 못 쓰 고 떨어지 는 건 참 을 수 있 지만 신발 안감 이 상할 것 같 은 느낌 의 양면 테이프 입니다 . 일단 부착 후 3 일 간 은 신지 않 을 예정 이 며 매일 신 는 신발 도 아니 기에 생각 보다 오래 쓸 수 도 있 겠 지만 다시 구매 할 생각 은 없 습니다 .',
 'negative\t" 루피 아니 야 " 우리 아이 말 이 에요 ㅠㅠ 루피 좋 아 하 는 우리 아이 , 항상 껴안 고 다녀 많이 헤졌 어요 . 그래서 최저 가 판매처 에서 두 개 더 구매 했 는데 기존 에 샀 던 두 개 랑 은 다른 디자인 이 네요 . 얼굴 은 이쁘 지만 , 전

In [22]:
def get_term_frequency(document):
    term_freq = {}

    words = document.split()

    for w in words:
        term_freq[w] = 1 + (0 if term_freq.get(w) is None else term_freq[w])
    
    return term_freq

In [29]:
get_term_frequency(lines[10])

{'negative': 1,
 '"': 2,
 '화이트': 1,
 '로': 1,
 '구매': 1,
 '했': 1,
 '는데': 1,
 '.': 2,
 '..': 2,
 '다른': 1,
 '제품': 1,
 '으로': 1,
 '배송': 1,
 '되': 1,
 '어': 1,
 '왔': 1,
 '어요': 1,
 '!': 2,
 '여기': 1,
 '가': 1,
 '외국': 1,
 '이': 1,
 '아니': 1,
 '라면': 1,
 '당장': 1,
 '반품': 1,
 '인데': 1,
 '~~~': 1}

In [30]:
def get_context_counts(lines, vocab, w_size=2):
    context_cnt = defaultdict(int)

    for line in lines:
        words = line.split()

        for i, w in enumerate(words):
            if w in vocab:
                for c in words[i - w_size:i + w_size]:
                    if w != c:
                        context_cnt[(w, c)] += 1
    return context_cnt

In [41]:
def get_co_occurrence_df(context_cnt, vocab):
    data = []

    for word1 in vocab:
        row = []

        for word2 in vocab:
            try :
                count = context_cnt[(word1, word2)]
            except KeyError:
                count = 0
            row.append(count)
            
        data.append(row)

    return pd.DataFrame(data, index=vocab, columns=vocab)

In [32]:
term_freq = pd.Series(
    get_term_frequency(' '.join(lines))
).sort_values(ascending=False)

term_freq

.             362139
고             208598
이             198967
하             171351
positive      170652
               ...  
방수형                1
퀄릴티도               1
ㅡㅡ더                1
ㅡㅡ주문상태확인하고         1
ㅜㅜ건반도              1
Length: 65614, dtype: int64

In [35]:
vector_size = 800

In [36]:
term_freq.index[:vector_size]

Index(['.', '고', '이', '하', 'positive', '도', '네요', '좋', 'negative', '에',
       ...
       '털', '밝', '치수', '건가요', '사료', '바랍니다', '누가', '확실히', '으려고', '품평'],
      dtype='object', length=800)

In [39]:
context_cnt = pd.Series(
    get_context_counts(
        lines,
        term_freq.index[:vector_size],
        w_size=4
    )
)

context_cnt

negative  !            63
!         negative    200
아서        negative    338
          !           179
          다           596
                     ... 
어여        v             1
ㅇㅇ        에요            4
          베이            2
          굿             3
          입니다           4
Length: 2615014, dtype: int64

In [42]:
df = get_co_occurrence_df(context_cnt, term_freq.index[:vector_size])

df

,.,고,이,하,positive,도,네요,좋,negative,에,...,털,밝,치수,건가요,사료,바랍니다,누가,확실히,으려고,품평
.,0,38014,49026,38350,3792,34475,59303,31834,4768,26418,...,281,283,202,177,177,746,282,282,192,210
고,37188,0,22532,55069,10332,75470,12892,29640,3595,16531,...,257,309,78,53,126,53,147,153,69,203
이,44145,21874,0,14743,3101,9912,26440,13216,4329,15657,...,442,210,38,116,167,58,83,224,55,218
하,35949,55007,16361,0,4253,21907,23599,14699,2146,11490,...,68,85,160,160,74,368,84,136,135,87
positive,971,1866,1136,981,0,178,1011,4305,0,802,...,1,22,1,0,4,0,0,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
바랍니다,747,79,56,391,1,21,15,24,1,25,...,0,0,0,2,0,0,0,1,0,1
누가,285,166,106,90,5,52,104,24,16,51,...,0,0,1,4,0,0,0,0,0,0
확실히,261,152,216,141,19,65,178,145,4,59,...,1,7,1,1,0,1,0,0,1,0
으려고,164,63,38,129,72,22,34,13,28,149,...,1,2,6,0,1,0,0,1,0,2


In [43]:
df.values.shape

(800, 800)

### Similarity Measures

In [44]:
import torch

In [52]:
def get_L1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()

In [46]:
def get_L2_distance(x1, x2):
    return ((x1 - x2) ** 2).sum() ** .5

In [47]:
def get_infinity_distance(x1, x2):
    return ((x1 - x2).abs()).max()

In [54]:
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / (((x1**2).sum() ** .5) * ((x2**2).sum() ** .5) + 1e-10)

In [50]:
def get_nearest(query, dataframe, metric, top_k, ascending=True):
    vector = torch.from_numpy(dataframe.loc[query].values).float()
    distances = dataframe.apply(
        lambda x: metric(vector, torch.from_numpy(x.values).float()),
        axis=1
    )
    top_distances = distances.sort_values(ascending=ascending)[:top_k]

    print(', '.join([f'{k} ({v:.1f})' for k, v in top_distances.items()]))

In [56]:
print('L1 distance:')
get_nearest('반품', df, get_L1_distance, 10)
print('\nL2 distance:')
get_nearest('반품', df, get_L2_distance, 10)
print('\nInfinity distance:')
get_nearest('반품', df, get_infinity_distance, 10)
print('\nCosine Similarity:')
get_nearest('반품', df, get_cosine_similarity, 10, ascending=False)

L1 distance:
반품 (0.0), 교환 (27890.0), ㅠㅠ (33880.0), ㅠ (34704.0), 말 (35802.0), ㅡㅡ (36658.0), 다시 (36880.0), 뭐 (37059.0), 진짜 (37214.0), 그리고 (37588.0)

L2 distance:
반품 (0.0), 교환 (3837.5), 다고 (4556.4), 확인 (4997.4), 다시 (5008.6), 깔끔 (5016.8), 불편 (5051.2), 말 (5077.9), 긴 (5153.3), 못 (5157.6)

Infinity distance:
반품 (0.0), 다고 (1858.0), 를 (1875.0), 긴 (1936.0), 라고 (2072.0), 여 (2176.0), 교환 (2215.0), 확인 (2258.0), 로 (2368.0), 깔끔 (2422.0)

Cosine Similarity:
반품 (1.0), 교환 (0.9), 환불 (0.9), 조립 (0.9), 그냥 (0.9), 불편 (0.9), 확인 (0.9), 사용 (0.9), 착용 (0.9), 기 (0.8)
